<a href="https://colab.research.google.com/github/jks5177/darknet/blob/master/%EB%9D%BC%EB%B2%A8%EB%A7%81%EC%A0%84%ED%8C%8C(3_26).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#기본 설정

In [ ]:
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!mkdir image

In [ ]:
!cp -r gdrive/MyDrive/CodingAlzi/Cow/* image/ #ID가 있는 소들

In [ ]:
!cp -r gdrive/MyDrive/CodingAlzi/cut_video /content/ #영상에서 자른 증가된 소 이미지

In [ ]:
!mkdir /content/cutting_img

#군집

In [ ]:
# for loading/processing the images
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

# models
from keras.applications.vgg16 import VGG16
from keras.models import Model

# clustering and dimeasion reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

# evaluate logistic regression fit on label spreading for semi-supervised learning
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelSpreading
from sklearn.linear_model import LogisticRegression

In [ ]:
#라벨이 있는 파일들을 경로 명으로 저장
label_path = "/content/image"
os.chdir(label_path)

label_cows = []

with os.scandir(label_path) as label_files :
  for label_file in label_files :
    if label_file.name.endswith('.jpg') :
      label_cows.append(label_file.name)

#라벨이 없는 파일들을 경로 명으로 저장
nolabel_path = "/content/cut_video"
os.chdir(nolabel_path)

nolabel_cows = []

with os.scandir(nolabel_path) as nolabel_folders :
  for nolabel_folder in nolabel_folders :
    with os.scandir(nolabel_folder) as nolabel_files :
      for nolabel_file in nolabel_files :
        if nolabel_file.name.endswith('.jpg') :
          nolabel_cows.append(nolabel_folder.name + "/" + nolabel_file.name)

In [ ]:
y_label = []
for label_cow in label_cows :
  name = label_cow.split("_")[1]
  y_label.append(int(name))

파일들이 잘 가져와 졌는지 확인했습니다.

In [ ]:
y_label

[492,
 398,
 495,
 489,
 487,
 496,
 479,
 487,
 496,
 495,
 398,
 479,
 439,
 501,
 489,
 487,
 439,
 501,
 492]

In [ ]:
len(label_cows)

19

In [ ]:
len(nolabel_cows)

5047

In [ ]:
nolabel_cows[:10]

['cow_501_1/1125-cow-0.jpg',
 'cow_501_1/1750-cow-0.jpg',
 'cow_501_1/1125-cow-7.jpg',
 'cow_501_1/5375-cow-0.jpg',
 'cow_501_1/3125-cow-2.jpg',
 'cow_501_1/5375-cow-11.jpg',
 'cow_501_1/4875-cow-3.jpg',
 'cow_501_1/5250-cow-24.jpg',
 'cow_501_1/1250-cow-18.jpg',
 'cow_501_1/5375-cow-14.jpg']

In [ ]:
name = list(map(str, y_label + nolabel_cows))

In [ ]:
# 특성을 추출할 모델을 부르고 이미지 전처리 후 특성 추출

#load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

def extract_features(file, model) :
  # load the image as a 224x224 array
  img = load_img(file, target_size=(224,224))
  # convert from 'PIL.Image.Image' to numpy array
  img = np.array(img)
  # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
  reshaped_img = img.reshape(1, 224, 224, 3)
  # prepare image for model
  imgx = preprocess_input(reshaped_img)
  # get the feature vector
  features = model.predict(imgx, use_multiprocessing=True)
  return features

In [ ]:
label_data = {}
os.chdir(label_path)
p = "/content/image/"

# lop through each image in the dataset
for cow in label_cows :
  # try to extract the geatures and update the dictionary
  try :
    feat = extract_features(cow, model)
    label_data[cow] = feat
  # if something ails, save the extracted features as a pickle file (optional)
  except :
    with open(p, 'wb') as file:
      pickle.dump(label_data, file)

nolabel_data = {}
os.chdir(nolabel_path)
p = "/content/cut_vidoe"

# lop through each image in the dataset
for cow in nolabel_cows :
  # try to extract the geatures and update the dictionary
  try :
    feat = extract_features(cow, model)
    nolabel_data[cow] = feat
  # if something ails, save the extracted features as a pickle file (optional)
  except :
    with open(p, 'wb') as file:
      pickle.dump(nolabel_data, file)

In [ ]:
nolabel_data

{'cow_501_1/1125-cow-0.jpg': array([[0.        , 0.        , 0.39522544, ..., 0.91991955, 0.10277563,
         1.5582314 ]], dtype=float32),
 'cow_501_1/1750-cow-0.jpg': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'cow_501_1/1125-cow-7.jpg': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.65949297]], dtype=float32),
 'cow_501_1/5375-cow-0.jpg': array([[0.        , 0.        , 0.        , ..., 0.        , 0.90287805,
         0.        ]], dtype=float32),
 'cow_501_1/3125-cow-2.jpg': array([[0.        , 0.        , 0.49331808, ..., 0.        , 1.5010598 ,
         0.        ]], dtype=float32),
 'cow_501_1/5375-cow-11.jpg': array([[0.        , 0.        , 0.        , ..., 0.        , 0.06287354,
         0.        ]], dtype=float32),
 'cow_501_1/4875-cow-3.jpg': array([[0.        , 0.        , 0.05198359, ..., 0.        , 0.        ,
         0.        ]], dtype=float32),
 'cow_501_1/5250-cow-24.jpg': array([[0.       , 0.       , 1.0466152

In [ ]:
# get a list of the filename
filenames = np.array(list(label_data.keys()))

In [ ]:
# 특징이 추출된 후 배열로 저장합니다.
label_feat = np.array(list(label_data.values()))
print(label_feat.shape)

nolabel_feat = np.array(list(nolabel_data.values()))
print(nolabel_feat.shape)

(19, 1, 4096)
(5047, 1, 4096)


In [ ]:
label_feat = label_feat.reshape(-1, 4096)
print(label_feat.shape)

nolabel_feat = nolabel_feat.reshape(-1, 4096)
print(nolabel_feat.shape)

(19, 4096)
(5047, 4096)


# Semi-Supervised Learning

In [ ]:
# evaluate logistic regression fit on label spreading for semi-supervised learning
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelSpreading
from sklearn.linear_model import LogisticRegression

In [ ]:
X_mixed = concatenate((label_feat, nolabel_feat))
nolabel = [-1 for _ in range(len(nolabel_feat))]
y_mixed = concatenate((y_label, nolabel))
print(X_mixed.shape)
print(y_mixed.shape)

(5066, 4096)
(5066,)


In [ ]:
y_mixed

array([492, 398, 495, ...,  -1,  -1,  -1])

In [ ]:
#LabelSpreading() 함수를 이용해서 라벨 전파를 실행하였습니다.
model = LabelSpreading()
model.fit(x, y_mixed)
tran_labels = model.transduction_

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


라벨전파가 얼마나 잘 됬는지 확인을 위해 데이터 프레임으로 작성 후 확인을 해봤습니다.

하지만 결과는 좋지 못하였습니다.

In [ ]:
dic = {}
dic[1] = tran_labels
df = pd.DataFrame(data=dic, index=name)
df = df.sort_index()
df.index

Index(['398', '398', '439', '439', '479', '479', '487', '487', '487', '489',
       ...
       'cow_501_9/8750-cow-0.jpg', 'cow_501_9/8875-cow-0.jpg',
       'cow_501_9/9000-cow-0.jpg', 'cow_501_9/9125-cow-0.jpg',
       'cow_501_9/9250-cow-0.jpg', 'cow_501_9/9375-cow-0.jpg',
       'cow_501_9/9500-cow-0.jpg', 'cow_501_9/9625-cow-0.jpg',
       'cow_501_9/9750-cow-0.jpg', 'cow_501_9/9875-cow-0.jpg'],
      dtype='object', length=5066)

In [ ]:
d= {}
d[1] = df.loc[['cow_501_9/9750-cow-0.jpg']].values
d[1] = np.append(d[1], 1)
d

{1: array([398,   1])}

In [ ]:
frame_dic = {}
for x in df.index :
  try:
    frame_name = x.split("/")[0] + "/" + x.split("/")[1].split("-")[0]
    if frame_name not in frame_dic.keys() :
      frame_dic[frame_name] = str(int(df.loc[[x]].values)) + "!" + '-'.join(x.split("/")[1].split("-")[1:])
      frame_dic[frame_name] = frame_dic[frame_name].values.tolist()
    else :
      frame_dic[frame_name] = np.append(frame_dic[frame_name], str(int(df.loc[[x]].values))+ "!" + '-'.join(x.split("/")[1].split("-")[1:]))
      frame_dic[frame_name] = sorted(frame_dic[frame_name])
  except:
    pass

In [ ]:
frame_dic

{'cow_479/1000': ['398!cow-0.jpg',
  '398!cow-11.jpg',
  '398!cow-13.jpg',
  '398!cow-18.jpg',
  '398!cow-3.jpg',
  '398!cow-9.jpg'],
 'cow_479/1125': ['398!cow-0.jpg', '398!cow-4.jpg', '398!cow-7.jpg'],
 'cow_479/125': ['398!cow-0.jpg', '398!cow-4.jpg'],
 'cow_479/1250': ['398!cow-0.jpg', '398!cow-1.jpg', '398!cow-4.jpg'],
 'cow_479/1375': ['398!cow-0.jpg', '398!cow-4.jpg', '398!cow-5.jpg'],
 'cow_479/1500': ['398!cow-0.jpg',
  '398!cow-10.jpg',
  '398!cow-13.jpg',
  '398!cow-14.jpg',
  '398!cow-20.jpg',
  '398!cow-8.jpg'],
 'cow_479/1625': ['398!cow-0.jpg',
  '398!cow-10.jpg',
  '398!cow-11.jpg',
  '398!cow-4.jpg',
  '398!cow-7.jpg'],
 'cow_479/1750': ['398!cow-0.jpg', '398!cow-4.jpg'],
 'cow_479/250': ['398!cow-0.jpg', '398!cow-2.jpg', '398!cow-6.jpg'],
 'cow_479/375': ['398!cow-0.jpg', '398!cow-1.jpg', '398!cow-7.jpg'],
 'cow_479/500': ['398!cow-0.jpg',
  '398!cow-11.jpg',
  '398!cow-14.jpg',
  '398!cow-2.jpg',
  '398!cow-8.jpg'],
 'cow_479/625': ['398!cow-0.jpg', '398!cow-4.jpg'],

In [ ]:
len(frame_dic.keys())

2314

In [ ]:
delete = []
for x in frame_dic.keys() :
  cow_id = []
  if type(frame_dic[x]) == list :
    for y in frame_dic[x] :
      if y.split("!")[0] in cow_id :
        delete.append(x+"-"+y.split("!")[1])
      else :
        cow_id.append(y.split("!")[0])

In [ ]:
delete[0:10]

['cow_479/1000-cow-11.jpg',
 'cow_479/1000-cow-13.jpg',
 'cow_479/1000-cow-18.jpg',
 'cow_479/1000-cow-3.jpg',
 'cow_479/1000-cow-9.jpg',
 'cow_479/1125-cow-4.jpg',
 'cow_479/1125-cow-7.jpg',
 'cow_479/125-cow-4.jpg',
 'cow_479/1250-cow-1.jpg',
 'cow_479/1250-cow-4.jpg']

In [ ]:
len(delete)

2733

#데이터저장

In [ ]:
dic = {}
for i in range(1, 4097) :
  pca = PCA(n_components=1, random_state=22)
  pca.fit(X_mixed)
  x = pca.transform(X_mixed)

  model = LabelSpreading()
  model.fit(x, y_mixed)
  tran_labels = model.transduction_

  dic[i] = tran_labels

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/usr/local/lib/pyth

In [ ]:
df = pd.DataFrame(data=dic, index=name)
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,...,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398,398
492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,...,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492
479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,...,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479,479
492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,...,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492,492
487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,...,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487,487


In [ ]:
df.to_csv('/content/result.csv')